# 원, 삼각형, 사각형 구분하기
## - 어려운 테스트 문제를 데이터 부풀리기를 이용한 경우

In [29]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
#층을 담는 그릇(Sequential은 keras에 포함되어 있는데
# 딥러닝 모델을 한층 한층 쌓기 쉽게 해주는 함수)
from tensorflow.keras.layers import Dense
#층을 만드는 것(각 층이 가질 특성을 각각 다르게 지정 가능) 
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

# seed값 생성
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

## 데이터셋 부풀리기

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.5,
    zoom_range=[0.8, 2.0],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [31]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

In [32]:
i = 0
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview',
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 20:
        break

In [33]:
train_generator = train_datagen.flow_from_directory(
    'hard_handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [34]:
test_datagen = ImageDataGenerator(rescale=1/255.)
test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


훈련시 데이터 부풀리기가 가능하지만 테스트 데이터는 부풀려서 할 수 없음

## 모델 정의/설정/학습

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [36]:
model = Sequential([ 
    Conv2D(32, (3,3), input_shape=(24,24,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

In [37]:

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## 모델 학습
케라스에서는 모델을 학습시킬 때 주로 fit() 함수를 사용하지만 제네레이터로 생성된 배치로 학습시킬 경우에는 fit_generator() 함수를 사용합니다. 본 예제에서는 ImageDataGenerator라는 제네레이터로 이미지를 담고 있는 배치로 학습시키기 때문에 fit_generator() 함수를 사용하겠습니다.

### 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 train_generator으로 지정합니다.
### steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정합니다. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정. 데이터 부풀리기를 이용하여 100개씩 사용
### epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정합니다. 100번을 반복적으로 학습시켜 보겠습니다.
### validation_data : 검증데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 validation_generator으로 지정합니다.
### validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정합니다. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정합니다.

In [38]:
model.fit(
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5,
        verbose=2
)

Epoch 1/200
15/15 - 0s - loss: 1.2012 - accuracy: 0.3333 - val_loss: 1.0848 - val_accuracy: 0.4667
Epoch 2/200
15/15 - 0s - loss: 1.0502 - accuracy: 0.4444 - val_loss: 1.0516 - val_accuracy: 0.4667
Epoch 3/200
15/15 - 0s - loss: 0.8591 - accuracy: 0.6222 - val_loss: 1.1808 - val_accuracy: 0.3333
Epoch 4/200
15/15 - 0s - loss: 0.8767 - accuracy: 0.5333 - val_loss: 0.9027 - val_accuracy: 0.6000
Epoch 5/200
15/15 - 0s - loss: 0.5727 - accuracy: 0.8222 - val_loss: 1.4032 - val_accuracy: 0.5333
Epoch 6/200
15/15 - 0s - loss: 0.3846 - accuracy: 0.8667 - val_loss: 1.9456 - val_accuracy: 0.5333
Epoch 7/200
15/15 - 0s - loss: 0.2487 - accuracy: 0.9111 - val_loss: 1.9282 - val_accuracy: 0.6000
Epoch 8/200
15/15 - 0s - loss: 0.2760 - accuracy: 0.8889 - val_loss: 1.6899 - val_accuracy: 0.5333
Epoch 9/200
15/15 - 0s - loss: 0.3270 - accuracy: 0.8889 - val_loss: 2.3651 - val_accuracy: 0.6000
Epoch 10/200
15/15 - 0s - loss: 0.1357 - accuracy: 0.9333 - val_loss: 2.2034 - val_accuracy: 0.6000
Epoch 11/

In [39]:
model.evaluate(test_generator, steps=5)

5/5 [==============================] - 0s 2ms/step - loss: 2.9907 - accuracy: 0.7333


[2.9906866550445557, 0.7333333492279053]